In [ ]:
%matplotlib inline
import os
data_dir = '/data/tutorials/nDarrays'

Multidimensional Arrays - Advanced Tutorial
======

# Outline

(sample xarray extensions)

1. EOF analysis on xarray objects ([`eofs`](http://ajdawson.github.io/eofs/index.html))
1. Combining xarray objects with vector shapes ([`regionmask`](http://regionmask.readthedocs.io/en/stable/index.html))

(xarray and big data)
1. Challenge: Out of core computation with [dask](https://dask.pydata.org/en/latest/)
2. Scaling out with [dask-distributed](http://distributed.readthedocs.io/en/latest/)

In [ ]:
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature


from eofs.xarray import Eof

import dask
from multiprocessing.pool import ThreadPool
dask.set_options(pool=ThreadPool(4))

# EOF analysis using xarray

In [ ]:
ds = xr.open_dataset(os.path.join(data_dir, 'SST_global.nc'))
ds.info()
sst = ds['sst']

In [ ]:
sst

In [ ]:
# resample to monthly
sst_monthly = sst.resample('MS', dim='time', how='mean')

In [ ]:
# calculate climatological anomalies
climatology = sst_monthly.groupby('time.month').mean('time')
anomalies = sst_monthly.groupby('time.month') - climatology

In [ ]:
# we'll subset the data to just the central/north Pacific
enso_34_region = dict(latitude=slice(62.5, -22.5),
                      longitude=slice(117.5, 262.5))
enso_34_anoms = anomalies.sel(**enso_34_region)

# we'll also select only the months between November and March
def month_in_ndjfm(month):
    if month >= 11 or month <= 3:
        return True
    return False

groups = xr.DataArray([month_in_ndjfm(t.month) for t in enso_34_anoms['time']],
                      dims='time', coords={'time': enso_34_anoms['time']}, name='ndjfm')

enso_34_anoms_ndjfm = enso_34_anoms.where(groups, drop=True)

In [ ]:
# sanity check, here's the first month in our dataarray
enso_34_anoms_ndjfm.isel(time=0).plot()

### Perform the EOF analysis on the temperature anomalies:

In [ ]:
# Create an EOF solver to do the EOF analysis. Square-root of cosine of
# latitude weights are applied before the computation of EOFs.
coslat = np.cos(np.deg2rad(enso_34_anoms_ndjfm.coords['latitude'].values))
wgts = np.sqrt(coslat)[..., np.newaxis]
solver = Eof(enso_34_anoms_ndjfm, weights=wgts)

# Retrieve the leading EOF, expressed as the correlation between the leading
# PC time series and the input SST anomalies at each grid point, and the
# leading PC time series itself.
eof1 = solver.eofsAsCorrelation(neofs=1)
pc1 = solver.pcs(npcs=1, pcscaling=1)

#### Visualize the EOFs/PCs

In [ ]:
# Plot the leading EOF expressed as correlation in the Pacific domain.
clevs = np.linspace(-1, 1, 11)
clevs = 11
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=215.25))
fill = eof1.sel(mode=0).plot.contourf(
    ax=ax, levels=clevs, cmap=plt.cm.RdBu_r,
    add_colorbar=False, transform=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, facecolor='w', edgecolor='k')
cb = plt.colorbar(fill, orientation='horizontal')
cb.set_label('correlation coefficient', fontsize=12)
ax.set_title('EOF1 expressed as correlation', fontsize=16)

def plot_classic_enso(pc1):
    fig, ax = plt.subplots()
    x = pc1.time.values
    y = pc1[:, 0].values

    ax.plot(x, y, color='k', linewidth=2)
    ax.fill_between(x, y, 0, where=(y >= 0), color='r', interpolate=True)
    ax.fill_between(x, y, 0, where=(y < 0), color='b', interpolate=True)

    ax.axhline(0, color='k')
    ax.set_ylim(-3, 3)
    ax.set_xlabel('Year')
    ax.set_ylabel('Normalized Units')
    ax.set_title('PC1 Time Series', fontsize=16)
    
    
# Plot the leading PC time series.
plot_classic_enso(pc1)

# pc1[:, 0].plot(color='b', linewidth=2)
# ax = plt.gca()


### Challenge 1: 

We just looked at the first EOF/PC. Explore these results by tweaking the a) the number of PCs, b) the analysis region, c) the time period.

# Masking using `regionmask`

`regionmask` is a package that when combined with xarray allows for easy masking / selecting of spatial regions in gridded datasets.

In [ ]:
import regionmask

In [ ]:
ds = xr.open_dataset(os.path.join(data_dir, 'airtemp_global.nc'))
KELVIN = 273.15
t2m = ds['t2m'].resample('AS', dim='time', how='mean') - KELVIN

masks  = regionmask.defined_regions.giorgi.mask(t2m,
                                                lat_name='latitude',
                                                lon_name='longitude',
                                                wrap_lon=True)

In [ ]:
t2m

In [ ]:
masks.plot(levels=masks.max(), cmap='tab20c')

In [ ]:
regionmask.defined_regions.giorgi.abbrevs

### get a mask-averaged timeseries for each of these masks

In [ ]:
import pandas as pd
df = pd.DataFrame(index=t2m.coords['time'])

for key in regionmask.defined_regions.giorgi.abbrevs:
    mask_num = regionmask.defined_regions.giorgi.map_keys(key)
    df[key] = t2m.where(masks == mask_num).mean(
        ('latitude', 'longitude')).to_series()
    
    

In [ ]:
df

In [ ]:
df.plot()

In [ ]:
def calc_slope(series):
    from scipy.stats.mstats import theilslopes
    x = series.index.year
    y = series.values
    
    medslope, _, lo, up = theilslopes(y)
    
    return medslope

df.apply(calc_slope)

# Challenge 2 - plot map of region slopes

# Challenge 3 - Use dask to repeat this analysis "out-of-core"

# Dask Distributed and Xarray

[`Dask.distributed`](https://distributed.readthedocs.io/en/latest/) is a lightweight library for distributed computing in Python. It extends both the concurrent.futures and dask APIs to moderate sized clusters.

xarray can use dask distributed in much the same way it uses dask. Here's a link to a wiki/screen-cast that describes how to use jupyter notebooks, dask distributed and xarray on a HPC environment.

https://github.com/pangeo-data/pangeo-discussion/wiki/Getting-Started-with-Dask-on-Cheyenne